<a href="https://colab.research.google.com/github/ulises1229/Intro-ML-Python/blob/master/code/D%C3%ADa_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Python y aplicaciones en Inteligencia Artificial
## Curso intersemestral enero 2022

---
### Dr. Jesús Emmanuel Solís Pérez
#### jessolisperez@gmail.com
---

# Seperación y entrenamiento

Para ajustar los modelos de regresión, hemos considerado toda la base de datos durante el entrenamiento del modelo. Como vimos con anterioridad esto puede presentar problemas de overfitting ante la presencia de datos nuevos. Para evitar este problema, vamos a dividir nuestra base de datos con lo siguiente:

`from sklearn.model_selection import train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split?

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import datasets

In [ ]:
boston_db = datasets.load_boston()

In [ ]:
boston_db['data'].shape

(506, 13)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston_db["data"], boston_db["target"],
                                                    test_size=0.30, random_state=1989) # 70% entrenamiento
                                                                                       # 30% prueba

In [ ]:
# Datos de entrenamiento
print(X_train.shape, y_train.shape)

# Datos de prueba
print(X_test.shape, y_test.shape)

(354, 13) (354,)
(152, 13) (152,)


In [ ]:
506*0.30 

151.79999999999998

In [ ]:
# 1. Creamos el modelo
mols = LinearRegression()

In [ ]:
# 2. Ajustamos el modelo con los datos de entrenamiento
mols.fit(X = X_train, y = y_train)

# 3. Realizamos las predicciones con los datos de entrenamiento
mols_train_pred = mols.predict(X_train)

In [ ]:
# 4. Evaluamos el modelo
def adjusted_R2(target, estimations, n, p):
    R2 = metrics.r2_score(target, estimations)
    return (R2 - ((p-1)/(n-p))*(1 - R2))

def eval_model(target, estimations, n, p):
    return {
        'RMSE': metrics.mean_squared_error(target, estimations),
        'R²': metrics.r2_score(target, estimations),
        'R²adj': adjusted_R2(target, estimations, n, p)
    }

In [ ]:
results = {}

In [ ]:
results['train'] = eval_model(y_train, mols_train_pred, X_train.shape[0], len(mols.coef_))

In [ ]:
pd.DataFrame(results)

,train
RMSE,19.115930
R²,0.764985
R²adj,0.756715


El modelo ya se encuentra entrenado. Los coeficientes del modelo de regresión lineal se encuentran en `modelo_ols.intercept_` y `modelo_ols.coef_`. Entonces vamos a utilizar el modelo con estos parámetros para evaluar su desempeño ante datos nuevos. 

In [ ]:
# 1. Utilizamos la base de datos de prueba para obtener las predicciones
mols_test_pred = mols.predict(X_test)

In [ ]:
# 2. Evaluamos el modelo
results['test'] = eval_model(y_test, mols_test_pred, X_test.shape[0], len(mols.coef_))

In [ ]:
pd.DataFrame(results)

,train,test
RMSE,19.115930,31.231160
R²,0.764985,0.658687
R²adj,0.756715,0.629221


¿Qué pasa si en lugar de utilizar `random_state=1989` usamos por ejemplo, `random_state=1999`?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston_db["data"], boston_db["target"],
                                                    test_size=0.30, random_state=1999)

In [ ]:
# 2. Ajustamos el modelo con los datos de entrenamiento
mols.fit(X = X_train, y = y_train)

# 3. Realizamos las predicciones con los datos de entrenamiento
mols_train_pred = mols.predict(X_train)

In [ ]:
# 4. Evaluamos el modelo
results['train_2'] = eval_model(y_train, mols_train_pred, X_train.shape[0], len(mols.coef_))

In [ ]:
# 1. Utilizamos la base de datos de prueba para obtener las predicciones
mols_test_pred = mols.predict(X_test)

In [ ]:
# 2. Evaluamos el modelo
results['test_2'] = eval_model(y_test, mols_test_pred, X_test.shape[0], len(mols.coef_))

In [ ]:
pd.DataFrame(results)

,train,test,train_2,test_2
RMSE,19.115930,31.231160,22.800213,20.546276
R²,0.764985,0.658687,0.740066,0.728408
R²adj,0.756715,0.629221,0.730919,0.704961


---
# Validación cruzada
El error varía dependiendo de la semilla utilizada al momento de hacer la separación de la base de datos en **prueba** y **entrenamiento**. Una forma de evitar esta variabilidad en los errores es utilizando la **validación cruzada**

<img src="pictures/cross_validation.jpg" />

En $k$ iteración se produce un error $e_{k}$. Al final, se puede obtener la media del error para obtener un índice final:

$$
 E = \frac{1}{k} \sum_{j=1}^{k} e_{k}.
$$

Para $k=4$, tenemos

$$
 E = \frac{1}{4} \left( e_{1} + e_{2} + e_{3} + e_{4} \right).
$$


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score?

https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
sorted(metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [ ]:
mols = LinearRegression()
Xdata = boston_db['data']
target = boston_db['target']

In [ ]:
results_cross_val_score = cross_val_score(estimator = mols, X = Xdata, y = target,
                                          scoring = "r2", cv = 10)

In [ ]:
results_cross_val_score

array([ 0.73376082,  0.4730725 , -1.00631454,  0.64113984,  0.54766046,
        0.73640292,  0.37828386, -0.12922703, -0.76843243,  0.4189435 ])

In [ ]:
results_cross_val_score.mean()

0.20252899006056085

---
# Práctica en clase

1. Considerar el **Ejemplo 1** del **Día 2** y dividir la base de datos en 60% entrenamiento y 40% prueba.
1. Considerar el **Ejemplo 2** del **Día 2** y dividir la base de datos en 60% entrenamiento y 40% prueba.